In [ ]:
import pennylane as qml
from entangler import get_entangler_map
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer

In [ ]:
n_wires = 8
n_weights = 3
dev = qml.device("default.qubit", wires=n_wires)

graph = get_entangler_map

In [ ]:
nb_layer_application = 3

In [ ]:
@qml.qnode(dev)
def circuit(weights, x=None):
    
    for i in range(nb_layer_application):
    
        circuit_feature_map(x)
        variational_circuit(weights[2*i:2*i+1])

    # Initializing observable
    diagonal = np.array(range(2**n_wires))
    avg_observable = np.zeros((2**n_wires, 2**n_wires))
    np.fill_diagonal(avg_observable, diagonal)

    return qml.expval(qml.Hermitian(avg_observable, list(range(n_wires))))

In [ ]:
# QAOA ansatz definition
# unitary operator U_B with parameter beta
def U_B(beta):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)

# unitary operator U_C with parameter gamma
def U_C(gamma):
    for wire_pair in graph:
        qml.CNOT(wires=[wire_pair[0], wire_pair[1]])
        qml.RZ(gamma, wires=wire_pair[1])
        qml.CNOT(wires=[wire_pair[0], wire_pair[1]])
        
def variational_circuit(weights):
    U_B(weights[0])
    U_C(weights[1])

In [ ]:
def loss(labels, predictions):
    return np.sum([(label - prediction)**2 for label, prediction in zip(labels, predictions)])

def cost(weights, X, Y):
    predictions = [circuit(weights, x=x) for x in X]
    return loss(Y, predictions)

In [ ]:
# Generate data
X = [[0,0], [1,1], [2,2]]
Y = [0, 1, 2]

In [ ]:
# Define optimizer. 
opt = GradientDescentOptimizer(stepsize=0.4)

# Setup initial weights and number of optimization steps.
weights = np.random.random(n_weights) * ( 2*np.pi )
steps = 300

# Optimizing the loss-function.
for it in range(steps):
    # Update the weights by one optimizer step
    weights = opt.step(lambda v: cost(v, X, Y), weights)